Primera Parte: Importación, exploración inicial y limpieza de datos

In [ ]:
# Primero importo todas las librerias necesarias y cargo el dataset
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# Carga del archivo
xls = pd.ExcelFile("Informe Regalias CRUDO.xlsx")

# Visualizo las hojas de cálculo para ver sobre las que voy a trabajar
print(xls.sheet_names)

Segunda Parte: Integración de datasets y creación del DataFrame final

In [ ]:
# Cargo la hoja de REGALIAS y limpio las columnas necesarias
df_regalias = pd.read_excel(xls, sheet_name='Tabla regalías', skiprows=11)
# Cargo la hoja de PRECIOS y selecciono AÑO, MES y PRECIO
df_precios = pd.read_excel(xls, sheet_name='Tabla precios', skiprows=12)
# Cargo la hoja de PRODUCCIÓN y selecciono AÑO, MES y PRODUCCIÓN
df_produccion = pd.read_excel(xls, sheet_name='Tabla producción', skiprows=11)

# Me quedo con las tres primeras columnas: AÑO, MES y REGALIAS
df_regalias_clean = df_regalias.iloc[:, [0, 1, 2]].copy()
df_regalias_clean.columns = ['AÑO', 'MES', 'REGALIAS']
df_regalias_clean.dropna(inplace=True)

# Me quedo con las tres primeras columnas: AÑO, MES y PRECIOS
df_precios_clean = df_precios.iloc[:, [0, 1, 2]].copy()
df_precios_clean.columns = ['AÑO', 'MES', 'PRECIO']
df_precios_clean.dropna(inplace=True)

# Me quedo con las tres primeras columnas: AÑO, MES y PRODUCCIÓN
df_produccion_clean = df_produccion.iloc[:, [0, 1, 2]].copy()
df_produccion_clean.columns = ['AÑO', 'MES', 'PRODUCCION']
df_produccion_clean.dropna(inplace=True)

In [ ]:
print("Tabla de Regalías")
display(df_regalias_clean.head())
print(" ")
print("Tabla de Precios")
display(df_precios_clean.head())
print(" ")
print("Tabla de Producción")
display(df_produccion_clean.head())

In [ ]:
# Conversión de columnas AÑO y MES a tipo entero para evitar problemas al unir los DataFrames
# Aseguro que las columnas AÑO y MES sean enteras para que el merge funcione correctamente
df_regalias_clean['AÑO'] = df_regalias_clean['AÑO'].astype(int)
df_regalias_clean['MES'] = df_regalias_clean['MES'].astype(int)

df_precios_clean['AÑO'] = df_precios_clean['AÑO'].astype(int)
df_precios_clean['MES'] = df_precios_clean['MES'].astype(int)

df_produccion_clean['AÑO'] = df_produccion_clean['AÑO'].astype(int)
df_produccion_clean['MES'] = df_produccion_clean['MES'].astype(int)

In [ ]:
# Uno regalías con precios
df_final = df_regalias_clean.merge(df_precios_clean, on=['AÑO', 'MES'], how='inner')

# Uno el resultado con producción
df_final = df_final.merge(df_produccion_clean, on=['AÑO', 'MES'], how='inner')

df_final.head()

In [ ]:
print("Columnas:", df_final.columns.tolist())
print("Cantidad de filas:", df_final.shape[0])
print("")
print("Regalías:", df_regalias_clean[['AÑO', 'MES']].drop_duplicates().shape[0])
print("Precios:", df_precios_clean[['AÑO', 'MES']].drop_duplicates().shape[0])
print("Producción:", df_produccion_clean[['AÑO', 'MES']].drop_duplicates().shape[0])

In [ ]:
# CARGA Y LIMPIEZA DE REGALÍAS, PRECIOS Y PRODUCCIÓN

# Hoja de Regalías
df_regalias = pd.read_excel(xls, sheet_name='Tabla regalías', skiprows=9)
df_regalias['AÑO'] = df_regalias['AÑO'].ffill()
df_regalias_clean = df_regalias.iloc[:, [0, 1, 2]].copy()
df_regalias_clean.columns = ['AÑO', 'MES', 'REGALIAS']
df_regalias_clean.dropna(inplace=True)

# Hoja de Precios
df_precios = pd.read_excel(xls, sheet_name='Tabla precios', skiprows=12)
df_precios['AÑO'] = df_precios['AÑO'].ffill()
df_precios_clean = df_precios.iloc[:, [0, 1, 2]].copy()
df_precios_clean.columns = ['AÑO', 'MES', 'PRECIO']
df_precios_clean.dropna(inplace=True)

# Hoja de Producción
df_produccion = pd.read_excel(xls, sheet_name='Tabla producción', skiprows=10)
df_produccion['AÑO'] = df_produccion['AÑO'].ffill()
df_produccion_clean = df_produccion.iloc[:, [0, 1, 2]].copy()
df_produccion_clean.columns = ['AÑO', 'MES', 'PRODUCCION']
df_produccion_clean.dropna(inplace=True)

In [ ]:
# Convierto AÑO y MES a enteros en los tres DataFrames
df_regalias_clean['AÑO'] = df_regalias_clean['AÑO'].astype(int)
df_regalias_clean['MES'] = df_regalias_clean['MES'].astype(int)

df_precios_clean['AÑO'] = df_precios_clean['AÑO'].astype(int)
df_precios_clean['MES'] = df_precios_clean['MES'].astype(int)

df_produccion_clean['AÑO'] = df_produccion_clean['AÑO'].astype(int)
df_produccion_clean['MES'] = df_produccion_clean['MES'].astype(int)

In [ ]:
# Unión de regalías con precios
df_final = df_regalias_clean.merge(df_precios_clean, on=['AÑO', 'MES'], how='inner')

# Unión con producción
df_final = df_final.merge(df_produccion_clean, on=['AÑO', 'MES'], how='inner')

# Vista preliminar
df_final.head()

In [ ]:
print("Cantidad de filas:", df_final.shape[0])
print("Columnas:", df_final.columns.tolist())

Tercera Parte: Análisis Exploratorio de Datos (EDA) y Entrenamiento del Modelo

In [ ]:
# Gráfico: Producción vs Regalías
plt.figure(figsize=(8, 5))
plt.scatter(df_final['PRODUCCION'], df_final['REGALIAS'], alpha=0.6)
plt.xlabel("Producción (m³)")
plt.ylabel("Regalías (ARS)")
plt.title("Relación entre Producción y Regalías")
plt.grid(True)
plt.show()
print("")
# Gráfico: Precio vs Regalías
plt.figure(figsize=(8, 5))
plt.scatter(df_final['PRECIO'], df_final['REGALIAS'], alpha=0.6, color='orange')
plt.xlabel("Precio del petróleo (ARS)")
plt.ylabel("Regalías (ARS)")
plt.title("Relación entre Precio y Regalías")
plt.grid(True)
plt.show()

In [ ]:
# Defino las variables independientes (X) y la dependiente (y)
X = df_final[['PRECIO', 'PRODUCCION']]
y = df_final['REGALIAS']

# Divido los datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creo y entreno el modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Realizo predicciones sobre los datos de prueba
y_pred = modelo.predict(X_test)

# Evaluo el modelo
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("R² Score:", r2)
print("RMSE:", rmse)

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(y_test, y_pred, alpha=0.6, color='green')
plt.xlabel("Regalías reales (ARS)")
plt.ylabel("Regalías predichas (ARS)")
plt.title("Comparación entre Regalías Reales y Predichas")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')
plt.grid(True)
plt.show()

Parte 4: Mejora del Modelo con Ridge

In [ ]:
# Creo y entreno el modelo Ridge
modelo_ridge = Ridge(alpha=1000000)
modelo_ridge.fit(X_train, y_train)

# Predicciones
y_pred_ridge = modelo_ridge.predict(X_test)

# Evaluación
r2_ridge = r2_score(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))

print("Ridge - R² Score:", r2_ridge)
print("Ridge - RMSE:", rmse_ridge)

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(y_test, y_pred_ridge, alpha=0.6, color='purple')
plt.xlabel("Regalías reales (ARS)")
plt.ylabel("Regalías predichas (ARS)")
plt.title("Ridge: Reales vs Predichas")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')
plt.grid(True)
plt.show()